In [ ]:
# DeepFaceLab: Basic Demo

In [1]:
%pip install deepface

  Using cached deepface-0.0.93-py3-none-any.whl.metadata (30 kB)
  Using cached numpy-2.3.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pandas-2.3.1-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached gdown-5.2.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached pillow-11.3.0-cp312-cp312-win_amd64.whl.metadata (9.2 kB)
  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Using cached keras-3.10.0-py3-none-any.whl.metadata (6.0 kB)
  Using cached flask-3.1.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached flask_cors-6.0.1-py3-none-any.whl.metadata (5.3 kB)
  Using cached mtcnn-1.0.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached retina_face-0.0.17-py3-none-any.whl.metadata (10 kB)
  Using cached fire-0.7.0-py3-none-any.whl
  Using cached gunicorn-23.0.0-py3-none-any.whl.metadata (4.4 kB)
  Using ca

In [7]:
%pip install pygame

  Using cached pygame-2.6.1-cp312-cp312-win_amd64.whl.metadata (13 kB)
Using cached pygame-2.6.1-cp312-cp312-win_amd64.whl (10.6 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip uninstall tensorflow

^C
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip uninstall tf-keras

^C
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


Note: you may need to restart the kernel to use updated packages.


Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.5.1%2Bcu121-cp312-cp312-win_amd64.whl (2449.3 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchvision-0.20.1%2Bcu121-cp312-cp312-win_amd64.whl (6.1 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.5.1%2Bcu121-cp312-cp312-win_amd64.whl (4.1 MB)
  Using cached https://download.pytorch.org/whl/networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached https://download.pytorch.org/whl/fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.13.1-py3-none-any.whl (6.2 MB)
  Using cached https://download.pytorch.org/whl/mpmath-1.3.0-py3-none-any.whl (536 kB)
Using cached https://download.pytorch.org/whl/fsspec-2024.6.1-py3-none-any.whl (177 kB)
Using cached https://download.pytorch.org/whl/networkx-3.3-py3-none-any.whl (1.7 MB)

   ---------------------------------------- 0/7 [mpmath]


In [1]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


CUDA available: True
GPU name: NVIDIA GeForce GTX 1070


In [12]:
import pygame
import cv2
from deepface import DeepFace
import numpy as np

# Setup PyGame window
pygame.init()
WIDTH, HEIGHT = 320, 240
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Real-Time Age/Gender/Emotion")
font = pygame.font.SysFont("Segoe UI", 12)

# Setup webcam
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, HEIGHT)

# Frame skipping for better performance
frame_skip = 3
frame_count = 0
analysis_result = []

clock = pygame.time.Clock()
running = True

DETECTOR_MODEL = "ssd"

while running:
    clock.tick(30)  # Target ~30 FPS
    frame_count += 1

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    ret, frame = cap.read()
    if not ret:
        continue

    # frame = cv2.resize(frame, (WIDTH, HEIGHT))


    # Run DeepFace analysis every few frames
    if frame_count % frame_skip == 0:
        try:
            analysis_result = DeepFace.analyze(
                frame,
                actions=['age', 'gender', 'emotion'],
                enforce_detection=False,
                detector_backend=DETECTOR_MODEL,
                silent=True
            )
        except Exception as e:
            print("Analysis error:", e)
            analysis_result = []

    # Display video in PyGame
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    surface = pygame.surfarray.make_surface(np.flip(np.rot90(rgb_frame), axis=0))
    screen.blit(surface, (0, 0))

    # Draw results
    if analysis_result:
        for face in analysis_result:
            region = face['region']
            x, y, w, h = region['x'], region['y'], region['w'], region['h']

            age = int(face['age'])
            emotion = face['dominant_emotion']
            gender_probs = face['gender']
            gender = (
                f"Female ({gender_probs['Woman']:.0f}%)"
                if gender_probs['Woman'] > gender_probs['Man']
                else f"Male ({gender_probs['Man']:.0f}%)"
            )
            label = f"{gender} | {age} y/o | {emotion}"

            pygame.draw.rect(screen, (0, 255, 0), pygame.Rect(x, y, w, h), 2)

            # Draw readable label
            text_surface = font.render(label, True, (255, 255, 255))
            text_rect = text_surface.get_rect()
            text_rect.centerx = x + w // 2
            text_rect.top = y - text_rect.height - 2  # a little gap above the box


            bg_surface = pygame.Surface((text_rect.width, text_rect.height))
            bg_surface.set_alpha(150)
            bg_surface.fill((0, 0, 0))
            screen.blit(bg_surface, text_rect)
            screen.blit(text_surface, text_rect)

    pygame.display.update()

cap.release()
pygame.quit()
